In [22]:
#
from bs4 import BeautifulSoup
import pymongo
import flask_mongoalchemy
from splinter import Browser
import requests as r
import time
import pandas as pd

In [84]:
#set up splinter browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)


# Current News from NASA Website

In [24]:
#visit url
url = "https://mars.nasa.gov/news/"
browser.visit(url)

#pull html text and parse
html_code = browser.html
soup = BeautifulSoup(html_code, "html.parser")

#grab needed info
news_title = soup.find('div', class_="content_title").text
news_p = soup.find('div', class_="article_teaser_body").text

In [25]:
print(news_title)
print("-"*30)
print(news_p)


Seven Ways Mars InSight is Different
------------------------------
NASA has a long and successful track record at Mars. Since 1965, it has flown by, orbited, landed and roved across the surface of the Red Planet. What can InSight -- planned for launch in May -- do that hasn’t been done before?


# Latest Featured Image

In [26]:
# Featured Image URL & visit
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

#navigate to link
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(10)
browser.click_link_by_partial_text('more info')

# #get html code once at page
image_html = browser.html

# #parse
soup = BeautifulSoup(image_html, "html.parser")

# #find path and make full path
image_path = soup.find('figure', class_='lede').a['href']
featured_image_url = "https://www.jpl.nasa.gov/" + image_path

In [27]:
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA18851_hires.jpg


# Mars Weather

In [28]:
marsweather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(marsweather_url)

weather_html = browser.html

soup = BeautifulSoup(weather_html, 'html.parser')

mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

In [29]:
print(mars_weather)

Sol 1970 (Feb 20, 2018), Sunny, high -16C/3F, low -77C/-106F, pressure at 7.32 hPa, daylight 05:38-17:26


# Mars Facts

In [30]:
#mars facts url and splinter visit
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

#get html
facts_html = browser.html

soup = BeautifulSoup(facts_html, 'html.parser')

In [31]:
#get the entire table
table_data = soup.find('table', class_="tablepress tablepress-id-mars")

In [32]:
#find all instances of table row
table_all = table_data.find_all('tr')

#set up lists to hold td elements which alternate between label and value
labels = []
values = []

#for each tr element append the first td element to labels and the second to values
for tr in table_all:
    td_elements = tr.find_all('td')
    labels.append(td_elements[0].text)
    values.append(td_elements[1].text)
        

In [33]:
#make a data frame and view
mars_facts_df = pd.DataFrame({
    "Label": labels,
    "Values": values
})

In [34]:
mars_facts_df

,Label,Values
0,Equatorial Diameter:,"6,792 km\n"
1,Polar Diameter:,"6,752 km\n"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)\n
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [35]:
# get html code for DataFrame
fact_table = mars_facts_df.to_html(header = False, index = False)
fact_table

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km\\n</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km\\n</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)\\n</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemisperes

In [96]:
# new url
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser.visit(usgs_url)

usgs_html = browser.html

soup = BeautifulSoup(usgs_html, "html.parser")

# gets class holding hemisphere picture
returns = soup.find('div', class_="collapsible results")
hemispheres = returns.find_all('div', class_="description")
# returns2 = returns.find_all('div', class_="description")
# hemispheres = returns2.a#find_all('a', class_="itemLink product-item")


#setup list to hold dictionaries
hemisphere_image_urls =[]

for a in hemispheres:
    #get title and link from main page
    title = a.h3
    link = "https://astrogeology.usgs.gov" + a.a['href']
    
    #follow link from each page
    browser.visit(link)
#     time.sleep(5)
    
    #get image links
    image_page = browser.html
    results = BeautifulSoup(image_page, 'html.parser')
    img_link = results.find('div', class_='downloads').find('li').a['href']
    
    # create image dictionary for each image and title
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = img_link
    
    hemisphere_image_urls.append(image_dict)
    
print(hemisphere_image_urls)
    
    #browser.visit


# usgs_html

[{'title': <h3>Cerberus Hemisphere Enhanced</h3>, 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': <h3>Schiaparelli Hemisphere Enhanced</h3>, 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': <h3>Syrtis Major Hemisphere Enhanced</h3>, 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': <h3>Valles Marineris Hemisphere Enhanced</h3>, 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [93]:
hemispheres[3].a['href']

'/search/map/Mars/Viking/valles_marineris_enhanced'

In [124]:
hemispheres[0].find('h3').text

'Cerberus Hemisphere Enhanced'

In [100]:
mars_dict = {
        "id": 1,
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "mars_weather": mars_weather,
        "fact_table": fact_table,
        "hemisphere_images": hemisphere_image_urls}

In [99]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': <h3>Cerberus Hemisphere Enhanced</h3>},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': <h3>Schiaparelli Hemisphere Enhanced</h3>},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': <h3>Syrtis Major Hemisphere Enhanced</h3>},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': <h3>Valles Marineris Hemisphere Enhanced</h3>}]

In [101]:
mars_dict['hemisphere_images']

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': <h3>Cerberus Hemisphere Enhanced</h3>},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': <h3>Schiaparelli Hemisphere Enhanced</h3>},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': <h3>Syrtis Major Hemisphere Enhanced</h3>},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': <h3>Valles Marineris Hemisphere Enhanced</h3>}]

In [122]:

# coding: utf-8

#
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
import requests
import time
import pandas as pd

def scrape():
    # # Current News from NASA Website

    #set up splinter browser
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless = False)

    #visit url
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)

    #pull html text and parse
    html_code = browser.html
    soup = BeautifulSoup(html_code, "html.parser")

    #grab needed info
    news_title = soup.find('div', class_="content_title").text
    news_p = soup.find('div', class_="article_teaser_body").text




    # # Latest Featured Image

    # Featured Image URL & visit
    jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(jpl_url)

    #navigate to link
    browser.click_link_by_partial_text('FULL IMAGE')
    time.sleep(20)
    browser.click_link_by_partial_text('more info')

    #get html code once at page
    image_html = browser.html

    #parse
    soup = BeautifulSoup(image_html, "html.parser")

    #find path and make full path
    image_path = soup.find('figure', class_='lede').a['href']
    featured_image_url = "https://www.jpl.nasa.gov/" + image_path




    # # Mars Weather

    #weather url and html
    marsweather_url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(marsweather_url)
    weather_html = browser.html

    #get lastest tweet
    soup = BeautifulSoup(weather_html, 'html.parser')
    mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text



    # # Mars Facts

    #mars facts url and splinter visit
    facts_url = "https://space-facts.com/mars/"
    browser.visit(facts_url)

    #get html
    facts_html = browser.html
    soup = BeautifulSoup(facts_html, 'html.parser')

    #get the entire table
    table_data = soup.find('table', class_="tablepress tablepress-id-mars")

    #find all instances of table row
    table_all = table_data.find_all('tr')

    #set up lists to hold td elements which alternate between label and value
    labels = []
    values = []

    #for each tr element append the first td element to labels and the second to values
    for tr in table_all:
        td_elements = tr.find_all('td')
        labels.append(td_elements[0].text)
        values.append(td_elements[1].text)
            

    #make a data frame
    mars_facts_df = pd.DataFrame({
        "Label": labels,
        "Values": values
    })


    # get html code for DataFrame
    fact_table = mars_facts_df.to_html(header = False, index = False)
    fact_table


    ### Hemisphere Images
    # new url
    usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

    browser.visit(usgs_url)

    usgs_html = browser.html

    soup = BeautifulSoup(usgs_html, "html.parser")

    # gets class holding hemisphere picture
    returns = soup.find('div', class_="collapsible results")
    hemispheres = returns.find_all('div', class_="description")

    #setup list to hold dictionaries
    hemisphere_image_urls =[]

    for a in hemispheres:
        #get title and link from main page
        titles = a.h3
        link = "https://astrogeology.usgs.gov" + a.a['href']
        
        #follow link from each page
        browser.visit(link)
        time.sleep(3)
        
        #get image links
        image_page = browser.html
        results = BeautifulSoup(image_page, 'html.parser')
        img_link = results.find('div', class_='downloads').find('li').a['href']
        # img_link = results.find('div', class_='downloads').find('li').a['href']
        
        # create image dictionary for each image and title
        image_dict = {}
        image_dict['title'] = titles
        image_dict['img_url'] = img_link
        
        hemisphere_image_urls.append(image_dict)
        
    #print(hemisphere_image_urls)

    mars_dict = {
        "id": 1,
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "mars_weather": mars_weather,
        "fact_table": fact_table,
        "hemisphere_images": hemisphere_image_urls
    }

    return mars_dict

print (mars_dict)


{'id': 1, 'news_title': 'Seven Ways Mars InSight is Different', 'news_p': 'NASA has a long and successful track record at Mars. Since 1965, it has flown by, orbited, landed and roved across the surface of the Red Planet. What can InSight -- planned for launch in May -- do that hasn’t been done before?', 'featured_image_url': 'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA18851_hires.jpg', 'mars_weather': 'Sol 1970 (Feb 20, 2018), Sunny, high -16C/3F, low -77C/-106F, pressure at 7.32 hPa, daylight 05:38-17:26', 'fact_table': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km\\n</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km\\n</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (

In [114]:
mars_dict2

NameError: name 'mars_dict2' is not defined